In [10]:
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, log_loss

In [11]:
param_array = pd.read_csv("parameters/parameter_array.csv").set_index("PARAM")

In [12]:
param_array

,AIRPORTS,LOOKAHEAD,DATA,CONFIG_SUPPORT,EPOCHS,NUMBER_TRIALS,PATIENCE
PARAM,,,,,,,
1,katl,1,PD,0,200,3,30
2,kclt,1,PD,0,200,3,30
3,kden,1,PD,0,200,3,30
4,kdfw,1,PD,0,200,3,30
5,kjfk,1,PD,0,200,3,30
...,...,...,...,...,...,...,...
716,kmem,12,PD_SM,1,200,3,30
717,kmia,12,PD_SM,1,200,3,30
718,kord,12,PD_SM,1,200,3,30


In [13]:
Param_Nums = []
DFs = []

Not_Finishes = []
for param in param_array.index:
    
    try:
        df = pd.read_csv(f"Results/Result_{param}/results_df.csv")
        Param_Nums.append(param)
        DFs.append(df)
        
        
    except:
        
        Not_Finishes.append(param)

    
df = pd.concat(DFs)
df["PARAM"] = Param_Nums
df = pd.merge(df, param_array, on="PARAM")

In [14]:
np.unique(df.LOOKAHEAD)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [15]:
df.columns

Index(['Unnamed: 0', 'param', 'train_loss', 'val_loss', 'test_loss',
       'train_accuracy', 'val_accuracy', 'test_accuracy', 'train_f1', 'val_f1',
       'test_f1', 'PARAM', 'AIRPORTS', 'LOOKAHEAD', 'DATA', 'CONFIG_SUPPORT',
       'EPOCHS', 'NUMBER_TRIALS', 'PATIENCE'],
      dtype='object')

In [16]:
df.groupby(["LOOKAHEAD","CONFIG_SUPPORT","DATA"]).mean()["test_loss"]

LOOKAHEAD  CONFIG_SUPPORT  DATA 
1          0               PD       0.031531
                           PD_SM    0.038326
                           SM       0.042580
           1               PD       0.031552
                           PD_SM    0.034196
                                      ...   
12         0               PD_SM    0.124198
                           SM       0.116195
           1               PD       0.107124
                           PD_SM    0.123356
                           SM       0.125173
Name: test_loss, Length: 72, dtype: float64

In [20]:
df.groupby(["CONFIG_SUPPORT","DATA"]).mean()["test_loss"]

CONFIG_SUPPORT  DATA 
0               PD       0.080298
                PD_SM    0.103856
                SM            NaN
1               PD       0.085078
                PD_SM    0.098941
                SM       0.098019
Name: test_loss, dtype: float64

In [18]:
param_array.loc[Not_Finishes]

,AIRPORTS,LOOKAHEAD,DATA,CONFIG_SUPPORT,EPOCHS,NUMBER_TRIALS,PATIENCE
PARAM,,,,,,,
502,kclt,3,SM,1,200,3,30
506,kmem,3,SM,1,200,3,30
509,kphx,3,SM,1,200,3,30
513,kden,4,SM,1,200,3,30
518,kord,4,SM,1,200,3,30
540,ksea,6,SM,1,200,3,30
542,kclt,7,SM,1,200,3,30
554,kdfw,8,SM,1,200,3,30
555,kjfk,8,SM,1,200,3,30
